In [1]:
#imports 
import SoccerNet
from SoccerNet.Downloader import SoccerNetDownloader
from SoccerNet.utils import getListGames
import os
import json
import pandas as pd


In [5]:

data_dir = "../SoccerNetData"
mySoccerNetDownloader = SoccerNetDownloader(LocalDirectory=data_dir)
mySoccerNetDownloader.password = "s0cc3rn3t"


In [10]:
games = getListGames(split="train")
print(f"Found {len(games)} training games.")
print("Example game:", games[0])

# ======================================
# 📥 5. Download raw videos (choose resolution)
# ======================================
# Download the lower-resolution videos (faster)
mySoccerNetDownloader.downloadGames(
    files=["1_224p.mkv", "2_224p.mkv"],
    split=["train"]
)

# OR download higher-quality 720p versions (larger files)
# mySoccerNetDownloader.downloadGames(
#     files=["1_720p.mkv", "2_720p.mkv"],
#     split=["train"]
# )

print(f"✅ Download complete. Files saved under: {data_dir}")

Found 300 training games.
Example game: england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley
../SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/1_224p.mkv already exists
../SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley/2_224p.mkv already exists
../SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/1_224p.mkv already exists
../SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal/2_224p.mkv already exists
../SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/1_224p.mkv already exists
../SoccerNetData/england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United/2_224p.mkv already exists
../SoccerNetData/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/1_224p.mkv already exists
../SoccerNetData/england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool/2_224p.mkv already exists
../

In [12]:
mySoccerNetDataLabels = SoccerNetDownloader(LocalDirectory="../SoccerNetLabels")
mySoccerNetDataLabels.password = "s0cc3rn3t"

# Download SoccerNet-v2 action spotting labels
mySoccerNetDataLabels.downloadGames(
    files=["Labels-v2.json"], 
    split=["train", "valid", "test"]
)

In [15]:

# Path to your SoccerNet root
audio_folder = "..SoccerNetDataAudio"
label_folder = "../SoccerNetLabels"
game = "2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht"

# Load the Labels-v2.json
labels_path = os.path.join(label_folder, game, "Labels-v2.json")
with open(labels_path, "r") as f:
    data = json.load(f)


annotations = data["annotations"]

# Example audio path
audio_dir = os.path.join(audio_folder, game)
audio_first_half = os.path.join(audio_dir, "1.wav")
audio_second_half = os.path.join(audio_dir, "2.wav")

print("Audio paths:")
print(audio_first_half)
print(audio_second_half)

# Extract labels for each half
events_first_half = [a for a in annotations if a["gameTime"].startswith("1 -")]
events_second_half = [a for a in annotations if a["gameTime"].startswith("2 -")]

# Show example
print("\nSample events for first half:")
for e in events_first_half[:5]:
    print(f"{e['label']} at {e['position']} sec ({e['visibility']})")


Audio paths:
..SoccerNetDataAudio/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/1.wav
..SoccerNetDataAudio/2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht/2.wav

Sample events for first half:
Kick-off at 23 sec (not shown)
Ball out of play at 59880 sec (visible)
Clearance at 83101 sec (visible)
Ball out of play at 122002 sec (not shown)
Throw-in at 128378 sec (not shown)


In [20]:
import os
import json
import pandas as pd

# --- Root directories ---
audio_root = "../SoccerNetDataAudio"
label_root = "../SoccerNetLabels"

# --- Helper: convert "1 - 00:00" → (half=1, seconds=0.0) ---
def parse_game_time(game_time_str):
    try:
        half, time_str = game_time_str.split(" - ")
        minutes, seconds = map(int, time_str.split(":"))
        total_seconds = minutes * 60 + seconds
        return int(half), total_seconds
    except Exception as e:
        print(f"⚠️ Failed to parse time '{game_time_str}': {e}")
        return None, None

# --- Step 1: find common games between audio + labels ---
audio_games = set(os.listdir(audio_root))
label_games = set(os.listdir(label_root))
common_games = sorted(list(audio_games.intersection(label_games)))

print(f"✅ Found {len(common_games)} matched games.\n")

rows = []

# --- Step 2: iterate over matched games ---
for game in common_games:
    audio_folder = os.path.join(audio_root, game)
    label_folder = os.path.join(label_root, game)
    label_path = os.path.join(label_folder, "Labels-v2.json")

    # Skip if label file missing
    if not os.path.exists(label_path):
        print(f"⚠️ Missing labels for {game}")
        continue

    wav_files = {}
    for f in os.listdir(audio_folder):
        if f.endswith(".wav"):
            prefix = f.split("_")[0]  # e.g. "1" from "1_224.wav"
            wav_files[prefix] = os.path.join(audio_folder, f)

    # Load JSON annotations
    with open(label_path, "r") as f:
        data = json.load(f)

    annotations = data.get("annotations", [])
    if not annotations:
        print(f"⚠️ No annotations found for {game}")
        continue

    # --- Step 3: match annotations to audio halves ---
    for ann in annotations:
        half, seconds = parse_game_time(ann.get("gameTime", ""))
        if half is None or half not in [1, 2]:
            continue

        wav_path = wav_files.get(str(half))
        if not wav_path:
            print(f"⚠️ Missing audio half {half} for {game}")
            continue

        # store record
        rows.append({
            "game": game,
            "half": half,
            "time_seconds": seconds,
            "label": ann.get("label", ""),
            "team": ann.get("team", ""),
            "visibility": ann.get("visibility", ""),
            "audio_path": wav_path
        })

# --- Step 4: convert to DataFrame ---
df = pd.DataFrame(rows)
print(f"✅ Created dataset with {len(df)} samples.")

# --- Step 5: preview ---
print(df.head())

# --- Step 6 (optional): save ---
output_csv = "../SoccerNet_audio_labels.csv"
df.to_csv(output_csv, index=False)
print(f"📁 Saved to {output_csv}")


✅ Found 300 matched games.

✅ Created dataset with 66460 samples.
                                          game  half  time_seconds  \
0  2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht     1             0   
1  2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht     1            59   
2  2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht     1            83   
3  2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht     1           122   
4  2014-11-04 - 22-45 Arsenal 3 - 3 Anderlecht     1           128   

              label            team visibility  \
0          Kick-off            home  not shown   
1  Ball out of play  not applicable    visible   
2         Clearance            away    visible   
3  Ball out of play  not applicable  not shown   
4          Throw-in            home  not shown   

                                          audio_path  
0  ../SoccerNetDataAudio/2014-11-04 - 22-45 Arsen...  
1  ../SoccerNetDataAudio/2014-11-04 - 22-45 Arsen...  
2  ../SoccerNetDataAudio/2014-11-04 - 22-45 Ars

In [6]:
df = pd.read_csv("../SoccerNet_audio_labels.csv")

# Get the last column name (audio path)
last_col = df.columns[-1]

# Replace "../SoccerNetDataAudio/" with the new path
df[last_col] = df[last_col].str.replace("../SoccerNetDataAudio/", "/storage/ice1/6/4/sviswanatha6/SoccerNetDataAudio/", regex=False)

# Save to a new file (or overwrite the old one)
df.to_csv("../SoccerNet_audio_labels.csv", index=False)